In [36]:
from elasticsearch import Elasticsearch
from tweepy.models import Status, ResultSet
import re
import json

In [71]:
friends_tweets = dict()
def extract_text(tweet):
    regex = r"http\S+"
    subset = ""
    emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)
    clean = re.sub(regex, subset, tweet)
    clean = emoji_pattern.sub(subset, clean).strip()
    return clean
def extract_hashtags(tweet):
    hashtags = []
    for k,v in tweet.items():
        if(k == 'hashtags' and v != None):
            for i in v:
                hashtags.append(i['text'])
    return hashtags

def extract_mentions(tweet):
    mentions = []
    for k,v in tweet.items():
        if(k == 'user_mentions' and v != None):
            for i in v:
                mentions.append(i['screen_name'])
    return mentions

es = Elasticsearch()
res = es.search(index="tweet", body={"query": {"match_all": {}}})
print("Got %d Hits:" % res['hits']['total']['value'])
for hit in res['hits']['hits']:
    friends_tweets[hash(str(hit["_source"]["user"]))] = {"text":[] , "mentions":[] , "hashtags" :[] }
    text = extract_text(hit["_source"]["text"])
    hashtags = extract_hashtags((hit["_source"]["entities"])) 
    mentions = extract_mentions((hit["_source"]["entities"]))
    friends_tweets[hash(str(hit["_source"]["user"]))]["text"].append(text)
    friends_tweets[hash(str(hit["_source"]["user"]))]["hashtags"].append(hashtags)
    friends_tweets[hash(str(hit["_source"]["user"]))]["mentions"].append(mentions)

Got 854 Hits:


In [72]:
with open('one_tweet.json', 'w') as f:
    json.dump(friends_tweets, f)